In [5]:
#
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import numpy as np
from PIL import Image
import os
import random

#
base_dir = "chest_xray"
splits = ["train", "validation", "test"]
categories = ["PNEUMONIA", "NORMAL"]

for split in splits:
    for category in categories:
        os.makedirs(os.path.join(base_dir, split, category), exist_ok=True)

#
def create_dummy_images(path, num_images=50):
    for i in range(num_images):
        img = np.random.randint(0, 256, (150, 150, 3), dtype=np.uint8)
        im = Image.fromarray(img)
        im.save(os.path.join(path, f"img_{i}.png"))

#
for category in categories:
    create_dummy_images(os.path.join(base_dir, "train", category), num_images=100)
    create_dummy_images(os.path.join(base_dir, "validation", category), num_images=40)
    create_dummy_images(os.path.join(base_dir, "test", category), num_images=40)

#
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, "train"),
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "validation"),
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary',
    shuffle=False
)

#
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

#
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    epochs=9,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//validation_generator.batch_size
)

#
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

Found 200 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Epoch 1/9
12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.5034 - loss: 11.8681 - val_accuracy: 0.5000 - val_loss: 1.8907
Epoch 2/9
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.4375 - loss: 14.2153 - val_accuracy: 0.5000 - val_loss: 2.0209
Epoch 3/9
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.5361 - loss: 6.9301 - val_accuracy: 0.5000 - val_loss: 2.5546
Epoch 4/9
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.6250 - loss: 3.5794 - val_accuracy: 0.5000 - val_loss: 1.9135
Epoch 5/9
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.4745 - loss: 3.4205 - val_accuracy: 0.5000 - val_loss: 0.8371
Epoch 6/9
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.4375 - loss: 4.4572 - val_accuracy: 0.5000 - val_loss: 0.8813
Epoch 7/9
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.5920 - loss: 2.0571 - val_accuracy: 0.5250 - val_loss: 0.861